In [ ]:
# ONLY RUN THIS CELL WHEN Selenium HASN'T ALREADY BEEN INSTALLED IN YOUR COMPUTER
!pip install selenium 

In [1]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [2]:
url = "https://www.youtube.com/results?search_query="
categories = ['travel','science','food','history','education']

In [12]:
def crawling(url:str, categories:list, num_of_pages:int = 0) -> pd.DataFrame:
    try:
        df = pd.DataFrame(columns = ['link','title','description','category'])
    
        for category in categories:
            print("crawling: " + category)
            crawling_url = url + category
            driver = webdriver.Chrome()
            driver.get(crawling_url)
            elements = driver.find_element_by_tag_name('body')
            while num_of_pages:
                elements.send_keys(Keys.PAGE_DOWN)  # scoll down to next pages
                time.sleep(0.5)
                num_of_pages -= 1
            
            #fetching all the link
            data = driver.find_elements_by_xpath('//*[@id="video-title"]')
            links = []
            for item in data:
                links.append(item.get_attribute('href'))
            
            # getting information
            wait = WebDriverWait(driver,10)
            count = 0
            for link in links:
                if not isinstance(link,str):
                    continue
                driver.get(link)
                id = link.strip('https://www.youtube.com/watch?v=')
                title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"h1.title yt-formatted-string"))).text
                des = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"div#description yt-formatted-string"))).text
                df.loc[len(df)] = [id, title,des,category]
                
                ###### for testing purposes
                count += 1
                if count > 0:
                    break
                ######
            time.sleep(1)
            driver.close()  # close tab automatically
            print("crawled totally {} link(s)".format(count))

        return df
    except Exception as ex:
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        print(message)
        print(ex)

In [13]:
df = crawling(url,[categories[1]])

crawling: science
crawled totally 1 links


In [7]:
df.to_csv("./sample.csv",encoding = 'utf-8')

In [8]:
d = pd.read_csv("./sample.csv",index_col = 0)

In [9]:
d.head(10)

,link,title,description,category
0,6LV3JdR5aZ,NaN,Our Social Media:\n\n5-Minute Crafts: https://...,science
